In [ ]:
import pandas as pd
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile

In [ ]:
def init_state_social_distancing_actions(path):
  df_master = pd.DataFrame()
  flag = False
  files_in_folder = glob.glob(path)
  for filename in files_in_folder:    
    zip_file = ZipFile(filename)
    for text_file in zip_file.infolist():
      # if not text_file.filename.startswith('__MACOSX/'):
      if text_file.filename.endswith('.csv'):
        date = re.search('\d*-\d*-\d*', text_file.filename)[0]
        if date == "20201-06-01":
          date = "2021-06-01"
        date_time_value = pd.to_datetime(date)
        df = pd.read_csv(zip_file.open(text_file.filename), sep=",", header=0)
        df["Date"] = date_time_value
        df.rename(columns = {'Unnamed: 0':'State'}, inplace = True)
        df.drop((df[df.State.isin(["United States"])].index) | (df[df.State.isnull()].index), inplace=True)
      if not flag:
        df_master = df
        flag = True
      else:
        df_master = pd.concat([df_master, df])
  df_master.set_index(["Date", "State"], inplace=True)
  df_master.sort_index(inplace=True)
  return df_master

state_social_distancing_actions = init_state_social_distancing_actions(r'**csv_files/state_social_distancing_actions.zip')

Clean up the dataframe to remove unused columns and solve for Nan fields

In [ ]:
def clean_state_social_distancing_actions(df):
  # TODO: Remove cells we don't want
  df = df.drop(columns=["Primary Election Postponement"])
  return df

cleaned_state_social_distancing_actions = clean_state_social_distancing_actions(state_social_distancing_actions)
cleaned_state_social_distancing_actions

Display 

In [ ]:
california_data = cleaned_state_social_distancing_actions[cleaned_state_social_distancing_actions.index.get_level_values('State').isin(['California'])]
california_data_mask_mandate = california_data[[ "Face Covering Requirement","Statewide Face Mask Requirement"]]
california_data_mask_mandate.values

# colors = sns.color_palette('pastel')
# plt.pie(california_data_mask_mandate.values, colors = colors)
# plt.show()

In [ ]:
def init_infection_dataframe_from_zip(path, index_col):
    df_master = pd.read_csv(glob.glob(path)[0], compression='zip', header=0, sep=',', index_col=index_col)
    return df_master

infected_df = init_infection_dataframe_from_zip(r'**csv_files/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv.zip', ["submission_date", "state"])

In [ ]:
infected_df.index = infected_df.index.set_levels([pd.to_datetime(infected_df.index.levels[0]), infected_df.index.levels[1]])
infected_df.sort_index()

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
#sns.lineplot(x = "submission_date", y = "tot_cases", data = infected_df)
plot = sns.lineplot(x = "submission_date", y = "new_case", data = infected_df[infected_df.index.get_level_values('state').isin(['CA'])])
plot.set(xlabel='Date', ylabel='Number of new cases')
plot.set_title('Number of new covid cases registered in California over time');
plt.xticks(rotation = 25);

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
#sns.lineplot(x = "submission_date", y = "tot_cases", data = infected_df)
plot = sns.lineplot(x = "submission_date", y = "tot_cases", data = infected_df[infected_df.index.get_level_values('state').isin(['CA'])])
plot.set(xlabel='Date', ylabel='Total cases (1e6)')
plot.set_title('Total number of covid cases registered in California over time');
plt.xticks(rotation = 25);